<a href="https://colab.research.google.com/github/yutao-data/Financial-Calculator/blob/main/investment_daily_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install tushare
!pip install yfinance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import tushare as ts
import os
from datetime import datetime, timedelta
import time

# Initialize the Tushare API
api_key_path = '/content/drive/My Drive/tushare_data/api_key.txt'
with open(api_key_path, 'r') as file:
    api_key = file.readline().strip()
pro = ts.pro_api(api_key)

# 股票市场

In [4]:
drive_path = '/content/drive/My Drive/tushare_data/status/stock_status'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

In [10]:
today = datetime.today().strftime('%Y%m%d')
yesterday = (datetime.today() - timedelta(1)).strftime('%Y%m%d')

def get_index_pe(index_code, date):
    index_data = pro.index_dailybasic(ts_code=index_code, trade_date=date, fields='ts_code,trade_date,pe')
    if not index_data.empty:
        pe = index_data.iloc[0]['pe']
        return pe
    return None

# 获取上证指数的市盈率
sh_pe = get_index_pe('000001.SH', today)
sh_pe = f"{sh_pe:.2f}" if sh_pe is not None else "暂无数据"

# 获取创业板指的市盈率
cyb_pe = get_index_pe('399006.SZ', today)
cyb_pe = f"{cyb_pe:.2f}" if cyb_pe is not None else "暂无数据"

# 获取两市成交数据
stock_data_today = pro.daily(trade_date=today)
stock_data_yesterday = pro.daily(trade_date=yesterday)

total_turnover_today = stock_data_today['amount'].sum() / 1e5  # 转换为亿元
total_turnover_yesterday = stock_data_yesterday['amount'].sum() / 1e5  # 转换为亿元
turnover_diff = total_turnover_today - total_turnover_yesterday

rise_count = (stock_data_today['pct_chg'] > 0).sum()  # 上涨家数
fall_count = (stock_data_today['pct_chg'] < 0).sum()  # 下跌家数

# 获取沪深港通资金流向数据
moneyflow_hsgt_today = pro.moneyflow_hsgt(trade_date=today)
moneyflow_hsgt_yesterday = pro.moneyflow_hsgt(trade_date=yesterday)

northbound_inflow_today = moneyflow_hsgt_today['north_money'].sum() / 1e2  # 转换为亿元
southbound_inflow_today = moneyflow_hsgt_today['south_money'].sum() / 1e2  # 转换为亿元

northbound_inflow_yesterday = moneyflow_hsgt_yesterday['north_money'].sum() / 1e2  # 转换为亿元
southbound_inflow_yesterday = moneyflow_hsgt_yesterday['south_money'].sum() / 1e2  # 转换为亿元

print(f"今日两市成交额：{total_turnover_today:.2f} 亿元, 较昨日{'增加' if turnover_diff > 0 else '减少'}：{abs(turnover_diff):.2f} 亿元, 上涨：{rise_count}家, 待涨：{fall_count}家, 北向资金净流入：{northbound_inflow_today:.2f} 亿元, 内资净流入：{southbound_inflow_today:.2f} 亿元, 上证指数市盈率：{sh_pe}，创业板指市盈率：{cyb_pe}。")

今日两市成交额：7261.35 亿元, 较昨日减少：603.74 亿元, 上涨：995家, 待涨：4193家, 北向资金净流入：0.00 亿元, 内资净流入：0.00 亿元, 上证指数市盈率：12.81，创业板指市盈率：25.70。


In [15]:
import yfinance as yf
from datetime import datetime, timedelta

# 定义指数代码
indices = {
    '标普500': '^GSPC',
    '纳斯达克': '^IXIC',
    '日经': '^N225',
    '恒生': '^HSI'
}

# 获取每个指数的今日数据
def get_index_data():
    data = {}

    for name, code in indices.items():
        ticker = yf.Ticker(code)
        hist = ticker.history(period='5d')  # 改为获取最近5天的数据
        if len(hist) >= 2:
            close = hist['Close'][-1]
            pre_close = hist['Close'][-2]
            change = close - pre_close
            pct_change = change / pre_close * 100
            trend = "上涨" if change > 0 else "下跌" if change < 0 else "持平"
            data[name] = (close, trend, pct_change)
        else:
            data[name] = None
    return data

# 输出趋势总结
def summarize_trends(data):
    summary = ""
    for name, values in data.items():
        if values:
            close, trend, pct_change = values
            summary += f"{name}今日收盘价为{close:.2f}点，相较昨日{trend}了{abs(pct_change):.2f}%。\n"
        else:
            summary += f"未能获取{name}的今日数据。\n"
    return summary

# 获取数据并打印输出
index_data = get_index_data()
print(summarize_trends(index_data))

<ipython-input-15-34652847de27>:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close = hist['Close'][-1]
<ipython-input-15-34652847de27>:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pre_close = hist['Close'][-2]
<ipython-input-15-34652847de27>:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close = hist['Close'][-1]
<ipython-input-15-34652847de27>:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecat

标普500今日收盘价为5312.75点，相较昨日下跌了2.46%。
纳斯达克今日收盘价为16619.42点，相较昨日下跌了3.34%。
日经今日收盘价为35909.70点，相较昨日下跌了5.81%。
恒生今日收盘价为16945.51点，相较昨日下跌了2.08%。



<ipython-input-15-34652847de27>:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  close = hist['Close'][-1]
<ipython-input-15-34652847de27>:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pre_close = hist['Close'][-2]


# 债券市场

In [6]:
drive_path = '/content/drive/My Drive/tushare_data/status/bond_status'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

In [7]:
today = datetime.today().strftime('%Y%m%d')
start_date = (datetime.today() - timedelta(days=5)).strftime('%Y%m%d')

# 获取国债收益率曲线数据
bond_yield_data = pro.yc_cb(ts_code='1001.CB', curve_type='0', start_date=start_date, end_date=today)
bond_yield_data = bond_yield_data.sort_values(by='trade_date', ascending=False)
bond_yield_data_10y = bond_yield_data[bond_yield_data['curve_term'] == 10]

if len(bond_yield_data_10y) > 1:
    bond_yield_today = bond_yield_data_10y.iloc[0]
    bond_yield_yesterday = bond_yield_data_10y.iloc[1]
    latest_bond_yield_today = bond_yield_today['yield']
    bond_yield_date_today = bond_yield_today['trade_date']
    latest_bond_yield_yesterday = bond_yield_yesterday['yield']
    bond_yield_date_yesterday = bond_yield_yesterday['trade_date']
    bond_yield_change = latest_bond_yield_today - latest_bond_yield_yesterday
else:
    latest_bond_yield_today = bond_yield_data_10y.iloc[0]['yield']
    bond_yield_date_today = bond_yield_data_10y.iloc[0]['trade_date']
    latest_bond_yield_yesterday = "暂无数据"
    bond_yield_date_yesterday = "暂无数据"
    bond_yield_change = "暂无数据"

# 获取最近的SHIBOR利率数据
shibor_data = pro.shibor(start_date=start_date, end_date=today)
shibor_data = shibor_data.sort_values(by='date', ascending=False)

if len(shibor_data) > 1:
    shibor_today = shibor_data.iloc[0]
    shibor_yesterday = shibor_data.iloc[1]
    latest_shibor_today = shibor_today['on']
    shibor_date_today = shibor_today['date']
    latest_shibor_yesterday = shibor_yesterday['on']
    shibor_date_yesterday = shibor_yesterday['date']
    shibor_change = latest_shibor_today - latest_shibor_yesterday
else:
    latest_shibor_today = shibor_data.iloc[0]['on']
    shibor_date_today = shibor_data.iloc[0]['date']
    latest_shibor_yesterday = "暂无数据"
    shibor_date_yesterday = "暂无数据"
    shibor_change = "暂无数据"

print(f"今日国债收益率（10年期）[{bond_yield_date_today}]：{latest_bond_yield_today}%，相较前一日（{bond_yield_date_yesterday}）变化：{bond_yield_change:.2f}%。")
print(f"最新上海银行间同业拆放利率（{shibor_date_today}）：{latest_shibor_today}%，相较昨日（{shibor_date_yesterday}）变化：{shibor_change:.2f}%。")


今日国债收益率（10年期）[20240801]：2.1297%，相较前一日（20240731）变化：-0.02%。
最新上海银行间同业拆放利率（20240802）：1.573%，相较昨日（20240801）变化：-0.04%。


# 美元兑人民币汇率

In [8]:
# 获取最近的外汇汇率数据
fx_rate_data = pro.fx_daily(ts_code='USDCNH.FXCM', start_date=start_date, end_date=today)
fx_rate_data = fx_rate_data.sort_values(by='trade_date', ascending=False)

if len(fx_rate_data) > 1:
    fx_rate_today = fx_rate_data.iloc[0]
    fx_rate_yesterday = fx_rate_data.iloc[1]
    latest_fx_rate_today = fx_rate_today['bid_close']
    fx_rate_date_today = fx_rate_today['trade_date']
    latest_fx_rate_yesterday = fx_rate_yesterday['bid_close']
    fx_rate_date_yesterday = fx_rate_yesterday['trade_date']
    fx_rate_change = latest_fx_rate_today - latest_fx_rate_yesterday
elif len(fx_rate_data) == 1:
    fx_rate_today = fx_rate_data.iloc[0]
    latest_fx_rate_today = fx_rate_today['bid_close']
    fx_rate_date_today = fx_rate_today['trade_date']
    latest_fx_rate_yesterday = "暂无数据"
    fx_rate_date_yesterday = "暂无数据"
    fx_rate_change = "暂无数据"
else:
    latest_fx_rate_today = "暂无数据"
    fx_rate_date_today = "暂无数据"
    latest_fx_rate_yesterday = "暂无数据"
    fx_rate_date_yesterday = "暂无数据"
    fx_rate_change = "暂无数据"

print(f"美元兑人民币汇率（{fx_rate_date_today}）：{latest_fx_rate_today}，相较昨日（{fx_rate_date_yesterday}）变化：{fx_rate_change*100:.3f}%。")

美元兑人民币汇率（20240801）：7.23316，相较昨日（20240731）变化：-1.708%。


# Financial News

In [9]:
drive_path = '/content/drive/My Drive/tushare_data/news'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

print("Fetching latest news from Sina...")
news_data = pro.news(src='sina', start_date='2024-07-21 09:00:00', end_date='2024-07-29 10:10:00')

print("Columns in news data:\n", news_data.columns)

random_news_data = news_data.sample(n=5)

print("Randomly selected news data sample:\n", random_news_data)

news_list = []

for index, row in random_news_data.iterrows():
    datetime_str = row.get('datetime', "暂无数据")
    title = row.get('title', "暂无数据") if pd.notna(row['title']) else "暂无数据"
    content = row.get('content', "暂无数据") if pd.notna(row['content']) else "暂无数据"

    news_desc = f"""
    时间：{datetime_str}
    标题：{title}
    内容：{content}
    """
    news_list.append(news_desc.strip())

print("\n最新的五个新闻：")
for news in news_list:
    print(news)
    print('-' * 80)

Fetching latest news from Sina...
Columns in news data:
 Index(['datetime', 'content', 'title'], dtype='object')
Randomly selected news data sample:
                 datetime                                            content  \
354  2024-07-28 18:37:48  【日本岩手县以东海域发生5.3级地震】据日本气象厅消息，当地时间7月28日19时13分左右，...   
78   2024-07-29 08:17:57                               标普：澳大利亚政府赤字仍然位居全球前列。   
603  2024-07-28 08:56:18  【中国制造的奥运纪念品卖爆了 粉色埃菲尔铁塔卖断货】巴黎奥运会期间，世界各地的游客在观看比赛...   
985  2024-07-27 13:31:32  【美瑞新材：聚氨酯产业园一期项目试生产方案通过验收】 美瑞新材7月26日接受机构调研时表示，...   
519  2024-07-28 12:13:54  【上海刚刚发布高温黄色预警】上海中心气象台12时发布高温黄色预警信号：受副热带高压影响，预计...   

    title  
354        
78         
603        
985        
519        

最新的五个新闻：
时间：2024-07-28 18:37:48
    标题：
    内容：【日本岩手县以东海域发生5.3级地震】据日本气象厅消息，当地时间7月28日19时13分左右，日本岩手县以东海域发生5.3级地震，最大震感为震度2，震源深度10公里。本次地震没有引发海啸的风险。
--------------------------------------------------------------------------------
时间：2024-07-29 08:17:57
    标题：
    内容：标普：澳大利亚政府赤字仍然位居全球前列。
------------------